In [1]:
#If th openai api key is not set already, please set it here.
import os
if 'OPENAI_API_KEY' not in os.environ:  
     os.environ['OPENAI_API_KEY'] = 'your key'

In [2]:
from amadeusgpt import AMADEUS
from amadeusgpt.config import Config
from amadeusgpt.utils import parse_result
import amadeusgpt
from pathlib import Path
import matplotlib.pyplot as plt
import cv2

### Note that unlike other notebooks, we don't have keypoint_file_path here (as it's not provided)
### By default, we use gpt-4o to determine which SuperAnimal models to run and it will run SuperAnimal in the first time the keypoints related queries are asked
### Make sure you use a short video clips if you are not using GPUs in Linux (Mac silicon support to be added)

In [4]:
scene_frame_number = 400
amadeus_root = Path(amadeusgpt.__file__).parent.parent
config = Config(amadeus_root / "amadeusgpt/configs/template.yaml")
config['data_info']['data_folder'] = "/Users/shaokaiye/OFT"

amadeus = AMADEUS(config)
video_file_paths = amadeus.get_video_file_paths()
print (video_file_paths)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


current total cost 0.004 $
current input tokens 25667
current accumulated tokens 25843
['/Users/shaokaiye/OFT/OFT_43_short.mp4']


In [ ]:
query = "Plot the trajectory of the animal using the animal center and color it by time"
qa_message = amadeus.step(query)
parse_result(amadeus, qa_message)

In [ ]:
query = "give me the occupancy plot using the animal center"
qa_message = amadeus.step(query)
qa_message = parse_result(amadeus, qa_message)